In [1]:
# Load necessary packages
import os
import gc
import h5py
import psutil
import pandas as pd
import scanpy as sc
from scipy import sparse

In [10]:
# Set-up the paths to data (TODO: change to your own paths)
input_dir = '/cellar/users/aklie/data/datasets/Zhu2023_sc-islet_snATAC-seq/processed/23Oct23/cellranger/MM168/outs'
h5_file = os.path.join(input_dir, 'filtered_peak_bc_matrix.h5')
bed_file = os.path.join(input_dir, 'peaks.bed')
output_dir = '/cellar/users/aklie/data/datasets/Zhu2023_sc-islet_snATAC-seq/annotation/24Oct23/scbasset/MM168'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
input_dir, h5_file, bed_file, output_dir

('/cellar/users/aklie/data/datasets/Zhu2023_sc-islet_snATAC-seq/processed/23Oct23/cellranger/MM168/outs',
 '/cellar/users/aklie/data/datasets/Zhu2023_sc-islet_snATAC-seq/processed/23Oct23/cellranger/MM168/outs/filtered_peak_bc_matrix.h5',
 '/cellar/users/aklie/data/datasets/Zhu2023_sc-islet_snATAC-seq/processed/23Oct23/cellranger/MM168/outs/peaks.bed',
 '/cellar/users/aklie/data/datasets/Zhu2023_sc-islet_snATAC-seq/annotation/24Oct23/scbasset/MM168')

# Load data

In [12]:
# Read peaks and h5ad
peak = pd.read_csv(bed_file, sep='\t', names=['chr','start','end'], comment='#')
ad = sc.read_10x_h5(h5_file, gex_only=False)
ad

AnnData object with n_obs × n_vars = 7998 × 192667
    var: 'gene_ids', 'feature_types', 'genome'

# Filter low quality cells

In [13]:
# Use Scanpy to process the data
ad.var['chr'] = peak['chr'].values
ad.var['start'] = peak['start'].values
ad.var['end'] = peak['end'].values

In [14]:
# basic stats
sc.pp.filter_cells(ad, min_genes=0)
sc.pp.filter_genes(ad, min_cells=0)

In [15]:
# a gene need to be expressed in 5% cells and a peak need to be accessible in 5% cells
thres = int(ad.shape[0]*0.05)
ad = ad[:, ad.var['n_cells'] > thres]

# Save h5ad for next step

In [16]:
# Keep only chromosomes 1-22, X, Y
chrs = ['chr'+str(i) for i in range(1,23)] + ['chrX', 'chrY']
ad = ad[:, ad.var['chr'].isin(chrs)]
ad.write(os.path.join(output_dir, 'atac_ad.h5ad'))

/cellar/users/aklie/opt/miniconda3/envs/scbasset/lib/python3.7/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/cellar/users/aklie/opt/miniconda3/envs/scbasset/lib/python3.7/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c


In [17]:
%%bash
source activate scbasset
python /cellar/users/aklie/opt/ml4gland/scBasset/bin/scbasset_preprocess.py \
    --ad_file /cellar/users/aklie/data/datasets/Zhu2023_sc-islet_snATAC-seq/annotation/24Oct23/scbasset/MM168/atac_ad.h5ad \
    --input_fasta /cellar/users/aklie/data/ml4gland/genomes/hg38/hg38.fa \
    --out_path /cellar/users/aklie/data/datasets/Zhu2023_sc-islet_snATAC-seq/annotation/24Oct23/scbasset/MM168

process 0 peaks takes 2.8 s
process 1000 peaks takes 5.5 s
process 2000 peaks takes 7.9 s
process 3000 peaks takes 9.8 s
process 4000 peaks takes 12.0 s
process 5000 peaks takes 14.7 s
process 6000 peaks takes 16.7 s
process 7000 peaks takes 19.0 s
process 8000 peaks takes 21.3 s
process 9000 peaks takes 23.1 s
process 10000 peaks takes 26.2 s
process 11000 peaks takes 31.4 s
process 12000 peaks takes 33.5 s
process 13000 peaks takes 35.9 s
process 14000 peaks takes 38.6 s
process 15000 peaks takes 42.1 s
process 16000 peaks takes 43.9 s
process 17000 peaks takes 46.2 s
process 18000 peaks takes 48.2 s
process 19000 peaks takes 50.8 s
process 20000 peaks takes 54.9 s
process 0 peaks takes 2.2 s
process 1000 peaks takes 3.2 s
process 2000 peaks takes 4.4 s
process 3000 peaks takes 6.1 s
process 4000 peaks takes 7.4 s
process 5000 peaks takes 8.4 s
process 6000 peaks takes 9.4 s
process 7000 peaks takes 10.1 s
process 8000 peaks takes 10.9 s
process 9000 peaks takes 11.8 s
process 10000 

# DONE!

---